# Before Threading:
<!-- ![f](../galleries/threading-1.svg) -->
<img src="/images/threading-1.svg" width="50%" align="left">  

# After Threading:
<img src="/images/threading-2.svg" width="50%" align="left">

# CPU Bound vs I/O bound

- A program is CPU bound if it would go faster if the CPU were faster, i.e. it spends the majority of its time simply using the CPU (doing calculations). A program that computes new digits of π will typically be CPU-bound, it's just crunching numbers.

- A program is I/O bound if it would go faster if the I/O subsystem was faster. Which exact I/O system is meant can vary; often associated with disk, but networking or communication in general is common too.

- Threading will improve I/O bound programs. Multiprocessing is needed to improve CPU bound programs

In [1]:
import requests
import time
import threading
import concurrent.futures

In [19]:
# verbose for example purposes. There are cleaner ways to do this below.
def do_something():
    print('Sleeping 1 second...')
    time.sleep(1)
    print('Done Sleeping...')

start = time.perf_counter()    

t1 = threading.Thread(target=do_something)
t2 = threading.Thread(target=do_something)

t1.start()
t2.start()

# join forces threads to finish before moving on to next lines of code
t1.join()
t2.join()

finish = time.perf_counter()

print(f'Finished in {round(finish-start,  2)} second(s)')


# We've run do_something (which takes at least  1 second) twice but since each run was given its own thread our total time is only around 1 second.

Sleeping 1 second...
Sleeping 1 second...
Done Sleeping...
Done Sleeping...
Finished in 1.0 second(s)


In [49]:
# using ThreadPoolExecutor in concurrent.futures module


def do_something(sec):
    print(f'Sleeping {sec} second...')
    time.sleep(sec)
    return f'Done Sleeping...{sec}'

start = time.perf_counter()

# use context manager with ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5, 4, 3, 2, 1]
    
    # This will give us a list of Future objects
    # A Future represents an eventual result of an asynchronous operation.
    futures = [executor.submit(do_something, sec) for sec in secs]
    
    # use as_completed function with result method to yield each future object as they are completed
    for f in concurrent.futures.as_completed(futures):
        # then use the result method to retrieve the return value
        print(f.result())
    
    # map method works like built in python map function
    # map returns returns the results in the order in which you pass iterables
    futures = executor.map(do_something, secs)
    print(list(futures))

finish = time.perf_counter()
print(f'Finished in {round(finish-start,  2)} second(s)')

Sleeping 5 second...Sleeping 4 second...

Sleeping 3 second...
Sleeping 2 second...
Sleeping 1 second...
Done Sleeping...1
Done Sleeping...2
Done Sleeping...3
Done Sleeping...4
Done Sleeping...5
Sleeping 5 second...
Sleeping 4 second...
Sleeping 3 second...
Sleeping 2 second...Sleeping 1 second...

['Done Sleeping...5', 'Done Sleeping...4', 'Done Sleeping...3', 'Done Sleeping...2', 'Done Sleeping...1']
Finished in 10.01 second(s)


In [2]:
img_urls = [
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
    'https://images.unsplash.com/photo-1524429656589-6633a470097c',
    'https://images.unsplash.com/photo-1530224264768-7ff8c1789d79',
    'https://images.unsplash.com/photo-1564135624576-c5c88640f235',
    'https://images.unsplash.com/photo-1541698444083-023c97d3f4b6',
    'https://images.unsplash.com/photo-1522364723953-452d3431c267',
    'https://images.unsplash.com/photo-1513938709626-033611b8cc03',
    'https://images.unsplash.com/photo-1507143550189-fed454f93097',
    'https://images.unsplash.com/photo-1493976040374-85c8e12f0c0e',
    'https://images.unsplash.com/photo-1504198453319-5ce911bafcde',
    'https://images.unsplash.com/photo-1530122037265-a5f1f91d3b99',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1550439062-609e1531270e',
    'https://images.unsplash.com/photo-1549692520-acc6669e2f0c'
]

In [4]:
def download_image(img_url):
    img_bytes = requests.get(img_url).content
    img_name = img_url.split('/')[3]
    img_name = f'{img_name}.jpg'
    with open(img_name, 'wb') as img_file:
        img_file.write(img_bytes)
        print(f'{img_name} was downloaded...')

In [51]:
# download with threading
t1 = time.perf_counter()

for img_url in img_urls:
    download_image(img_url)
    
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

photo-1516117172878-fd2c41f4a759.jpg was downloaded...
photo-1532009324734-20a7a5813719.jpg was downloaded...
photo-1524429656589-6633a470097c.jpg was downloaded...
photo-1530224264768-7ff8c1789d79.jpg was downloaded...
photo-1564135624576-c5c88640f235.jpg was downloaded...
photo-1541698444083-023c97d3f4b6.jpg was downloaded...
photo-1522364723953-452d3431c267.jpg was downloaded...
photo-1513938709626-033611b8cc03.jpg was downloaded...
photo-1507143550189-fed454f93097.jpg was downloaded...
photo-1493976040374-85c8e12f0c0e.jpg was downloaded...
photo-1504198453319-5ce911bafcde.jpg was downloaded...
photo-1530122037265-a5f1f91d3b99.jpg was downloaded...
photo-1516972810927-80185027ca84.jpg was downloaded...
photo-1550439062-609e1531270e.jpg was downloaded...
photo-1549692520-acc6669e2f0c.jpg was downloaded...
Finished in 82.98283140000422 seconds


In [52]:
# download with threading
t1 = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(download_image, img_urls)

t2 = time.perf_counter()

print(f'Finished in {t2-t1} seconds')

photo-1522364723953-452d3431c267.jpg was downloaded...
photo-1524429656589-6633a470097c.jpg was downloaded...
photo-1493976040374-85c8e12f0c0e.jpg was downloaded...
photo-1507143550189-fed454f93097.jpg was downloaded...
photo-1516117172878-fd2c41f4a759.jpg was downloaded...
photo-1549692520-acc6669e2f0c.jpg was downloaded...
photo-1564135624576-c5c88640f235.jpg was downloaded...
photo-1550439062-609e1531270e.jpg was downloaded...
photo-1516972810927-80185027ca84.jpg was downloaded...
photo-1530224264768-7ff8c1789d79.jpg was downloaded...
photo-1504198453319-5ce911bafcde.jpg was downloaded...
photo-1530122037265-a5f1f91d3b99.jpg was downloaded...
photo-1532009324734-20a7a5813719.jpg was downloaded...
photo-1513938709626-033611b8cc03.jpg was downloaded...
photo-1541698444083-023c97d3f4b6.jpg was downloaded...
Finished in 67.00409320000472 seconds
